# IBM Applied Capstone Week 3 assessment 
## Clustering Neighborhoods in Toronto
##### First I start by importing the requests library

In [191]:
import requests

###### Then I make a call to the wikipedia webpage to extract html data from it

In [192]:
Wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')


##### Using the beautiful soup library I can parse the html data

In [193]:
from bs4 import BeautifulSoup
Wikisoup = BeautifulSoup(Wiki.content, 'html.parser')
Wikisoup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","

##### I notice that the html data pertaining to the table begins where the table tag with class = "wikitable sortable" starts so I create a variable 'table' that will store the children within that parent tag

In [194]:
table= Wikisoup.find(class_= 'wikitable sortable')


##### I then make a list, 'header_names' where I extract the three table headers into using the get_text() function and a for loop

In [195]:
headers = table.find_all('th')
header_names = [tag.get_text() for tag in headers]
header_names

['Postcode', 'Borough', 'Neighbourhood\n']

##### In the next block of code I extract the data from each row in the table and store in a list of lists, info_cleaner

In [196]:
rows = table.find_all('tr') 
data_rows = rows[1:]
info = [rows.get_text() for rows in data_rows]
info_clean = [item.split('\n') for item in info]
info_cleaner = [item[1:4] for item in info_clean]



In [197]:
header_names[2] = 'Neighborhood'
header_names[0] = 'PostalCode'

In [198]:
header_names

['PostalCode', 'Borough', 'Neighborhood']

##### Then I plug my extracted data into a pandas dataframe, toronto, with my header_names as column headings and data from info_cleaner as my rows

In [199]:
import pandas as pd
Master = pd.DataFrame(data = info_cleaner, columns = header_names)
toronto = Master[Master.Borough != 'Not assigned']
toronto = toronto.groupby('PostalCode', as_index = False).agg({'Neighborhood': lambda x: ', '.join(x),'Borough': 'min'})

###### I replace the Neighbourhood entries that are 'Not assigned' with the corresponding borough name for that neighbourhood using this next line of code - this took ALOT of skull cracking to figure out! Pheww!

In [200]:
toronto['Neighborhood'] = toronto['Neighborhood'].replace('Not assigned', toronto['Borough'])


###### Let us see how many rows of data is in this dataframe

In [201]:
print(toronto.shape)

(103, 3)


###### 'import geocoder' keeps throwing up an errors so I decide to read the csv file containing latitude and longitude data of each postal code into a Pandas dataframe. I then merge the dataframe with the toronto dataframe that contains data on the Neighbourhoods and Boroughs.

In [202]:

!wget -O 'geospatial.csv' http://cocl.us/Geospatial_data #importing the csv file as 'geospatial.csv'



--2019-03-09 01:30:50--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-03-09 01:30:50--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-09 01:30:52--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-09 01:30:53--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [203]:
import pandas as pd
geospatial = pd.read_csv('geospatial.csv') #Reading the csv file into a pandas data frame


###### Merging the two dataframes 

In [204]:
toronto_merged = pd.concat([toronto, geospatial], axis=1)
toronto_merged = toronto_merged.drop('Postal Code', axis=1)
toronto_merged = toronto_merged[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
toronto_merged


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## <font color = blue > Let us get the additional packages that we will need for plotting </font_color>
##### We will specifically need the following: 
 - Folium - for interactive geospatial maps 

In [205]:
import matplotlib.cm as cm 
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


##### To create a map of toronto with the folium package, I will need to use the Nominatim library to get the lat, lng coordinates of Toronto, CA - super efficient tool no? 

In [206]:
address='Toronto, CA'

from geopy.geocoders import Nominatim # importing Nominatim 
geolocator=Nominatim(user_agent='ca_explorer')
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

###### Here the fun begins as I create the map of toronto and use a for loop to populate the map with circle indicators that label the neighbourhoods and boroughs on the map! I feel like I have a superpower now that I am able to do this! Thanks Coursera :D 

In [207]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat,lng, neighborhood, borough in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Borough']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius= 5,
        popup =label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

###### From this point onwards, I will restrict my analysis to the Neighborhoods within the 'Downtown Toronto' borough. The next line of code creates a new dataframe, 'downtown' which contains data only for the neighbourhoods in that demographic 

In [208]:
downtown = toronto_merged[toronto_merged['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


# _And without further ado, the analysis begins...._

###### Credentials for Foursqaure API 

In [209]:
CLIENT_ID = 'UZE23ZRN01VBZLPTHRWDU2MU4M2KZ5BXZ4FCKM4LUTWPZP0W'
CLIENT_SECRET = 'IEA40OERF4RWGORV3UWOT1MQFOYXG00PHFKPG4YNT5KJMYVK'
VERSION = '20180605'

##### I start by defining a function that will enable me to get data on the venues for each neighbourhood in Downtown Toronto 

In [210]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [211]:
downtown_venues = getNearbyVenues(names=downtown['Neighbourhood'],latitudes=downtown['Latitude'],longitudes=downtown['Longitude'])

KeyError: 'Neighbourhood'

###### How many venues in total do we get?

In [ ]:
print(downtown_venues.shape[0])

###### What does the dataframe look like?

In [ ]:
downtown_venues.head()

###### How many unique venues can we find in Downtown Toronto?

In [ ]:
print('There are {} unique venues in Downtown Toronto'.format(len(downtown_venues['Venue Category'].unique())))

###### Now shall we take a peek into the most popular venue categories across all the neighbourhoods

In [ ]:
categories = downtown_venues.groupby('Venue Category').count()
categories = categories.sort_values(by = 'Neighborhood', ascending=False)
categories

In [ ]:
print('In Downtown Toronto, there are about {} coffee shops'.format(categories.loc['Coffee Shop','Neighborhood']))

###### What's with Downtown Torontonians and coffee huh?

In [ ]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot

In [ ]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').sum().reset_index()
downtown_grouped

###### Let us organize the data from downtown_grouped so that we get a list of top 5 venues within each neighborhood. 

In [ ]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

###### Here I will create a dataframe which will indicate the top 10 most common venues for each neigborhood in Downtown Toronto

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np #importing numpy library
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

###### I am now going to cluster the neigborhoods! I use 3 clusters because my dataset is small ~ 18 neighborhoods


In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


##### I want to be able to see the cluster label for each neighborhood

In [214]:
downtown_merged = downtown
neighborhoods_venues_sorted.insert(0, 'Clusters', kmeans.labels_)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
downtown_merged # check the last columns! 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Park,Pizza Place,Pub,Café,Chinese Restaurant,Diner
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Gastropub,Café,Burger Joint
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Shoe Store,Brewery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Burger Joint,Bar,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Pizza Place
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Clothing Store,Gastropub,Park,Cocktail Bar,Cosmetics Shop
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Steakhouse,Pub,Restaurant,Café,Cheese Shop,Italian Restaurant,Seafood Restaurant
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Café,Bar,Burger Joint,Bubble Tea Shop,Salad Place,Sandwich Place,Spa,Chinese Restaurant
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Asian Restaurant,Hotel,American Restaurant,Bakery,Restaurant
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,2,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Pizza Place,Café,Restaurant,Scenic Lookout,Brewery,History Museum


In [215]:
address = 'Downtown Toronto, CA'

geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

###### Time to visualize the clusters! 

In [217]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters